# Sharding with jax

Hicham Randrianarivo

# Sharding with jax

<a
href="https://colab.research.google.com/github/chicham/chicham.github.io/blob/master/posts/sharding_jax/sharding_jax.ipnb"
data-fig-align="left"><img
src="https://colab.research.google.com/assets/colab-badge.svg" /></a>

## Links

-   [Tutorial ICML
    2022](https://sites.google.com/view/icml-2022-big-model/home)
-   [Distributed arrays and automatic
    parallelization](https://jax.readthedocs.io/en/latest/notebooks/Distributed_arrays_and_automatic_parallelization.html)
-   [https://github.com/google-research/big_vision](Big%20vision) the
    idea of the sharding class

## Pre-requisites

-   Train a 8 layers model
-   Generate random data cube

## Methods

### Baseline

-   Data parallelism
    -   Data parallelism with all reduce
-   Model Parallelism
    -   Device placement
        -   issue: each modules must wait for the computation of the
            previous one
        -   Micro batches
            -   GPipe
            -   1F1B
            -   Interleaved 1F1B (megatron)
                -   Split layers between multiple devices
        -   Async pipelines (won’t cover this)
    -   Split parameters ( intra-op parallelism)
        -   Megatron-LM
        -   ZeRO